# **Fashion MNIST: Data Normalization**

***
***

In [1]:
import os
from google.cloud import storage

# Create a local directory for your files
os.makedirs(os.path.expanduser("~/data_normalizer"), exist_ok=True)

# Download the files from GCS
client = storage.Client()
bucket = client.bucket("fashion-mnist-dev")

# Download main.py
blob = bucket.blob("data-normalizer/main.py")
blob.download_to_filename(os.path.expanduser("~/data_normalizer/main.py"))

# Download requirements.txt
blob = bucket.blob("data-normalizer/requirements.txt")
blob.download_to_filename(os.path.expanduser("~/data_normalizer/requirements.txt"))

print("Files downloaded successfully to ~/data_normalizer/")

Files downloaded successfully to ~/data_normalizer/


In [2]:
import subprocess
subprocess.run(["pip", "install", "-r", os.path.expanduser("~/data_normalizer/requirements.txt")])

CompletedProcess(args=['pip', 'install', '-r', '/home/jupyter/data_normalizer/requirements.txt'], returncode=0)

In [3]:
# Create a simple runner script
with open(os.path.expanduser("~/data_normalizer/run.py"), "w") as f:
    f.write("""
import sys
sys.path.append("/home/jupyter/data_normalizer")

from main import normalize_fashion_mnist
import json

# Define your input and output paths
input_path = "gs://fashion-mnist-datasets/custom_jobs/fashion_mnist.npz"
output_path = "gs://fashion-mnist-datasets/custom_jobs_normalized/"

# Run the normalization function
result = normalize_fashion_mnist(input_path, output_path)
print(f"Normalization result: {json.dumps(result, indent=2)}")
""")

print("Runner script created at ~/data_normalizer/run.py")

Runner script created at ~/data_normalizer/run.py


In [4]:
import subprocess
subprocess.run(["python", os.path.expanduser("~/data_normalizer/run.py")])

2025-04-28 19:17:13,467 - main - INFO - Downloading gs://fashion-mnist-datasets/custom_jobs/fashion_mnist.npz to /var/tmp/tmpcrtwd2tx/fashion_mnist.npz
2025-04-28 19:17:14,053 - main - INFO - Download complete
2025-04-28 19:17:14,053 - main - INFO - Loading dataset from /var/tmp/tmpcrtwd2tx/fashion_mnist.npz
2025-04-28 19:17:14,417 - main - INFO - Dataset loaded with keys: dict_keys(['X_train', 'y_train', 'X_test', 'y_test'])
2025-04-28 19:17:14,417 - main - INFO - Normalizing dataset
2025-04-28 19:17:14,512 - main - INFO - Normalized X_train: shape=(60000, 28, 28), dtype=float32
2025-04-28 19:17:14,513 - main - INFO - Kept y_train unchanged: shape=(60000,), dtype=uint8
2025-04-28 19:17:14,530 - main - INFO - Normalized X_test: shape=(10000, 28, 28), dtype=float32
2025-04-28 19:17:14,530 - main - INFO - Kept y_test unchanged: shape=(10000,), dtype=uint8
2025-04-28 19:17:14,530 - main - INFO - Saving normalized dataset to /var/tmp/tmpcrtwd2tx/fashion_mnist_normalized.npz
2025-04-28 19:1

Normalization result: {
  "status": "success",
  "input_path": "gs://fashion-mnist-datasets/custom_jobs/fashion_mnist.npz",
  "output_path": "gs://fashion-mnist-datasets/custom_jobs_normalized/",
  "normalized_file": "gs://fashion-mnist-datasets/custom_jobs_normalized/fashion_mnist_normalized.npz",
  "timestamp": "2025-04-28T19:17:23.926234"
}


2025-04-28 19:17:23,926 - main - INFO - Upload complete
2025-04-28 19:17:23,926 - main - INFO - Created and uploaded README.md to gs://fashion-mnist-datasets/custom_jobs_normalized/README.md


CompletedProcess(args=['python', '/home/jupyter/data_normalizer/run.py'], returncode=0)

In [5]:
subprocess.run(["gsutil", "ls", "-l", "gs://fashion-mnist-datasets/custom_jobs_normalized/"])

      1058  2025-04-28T19:17:23Z  gs://fashion-mnist-datasets/custom_jobs_normalized/README.md
       106  2025-04-28T19:17:23Z  gs://fashion-mnist-datasets/custom_jobs_normalized/class_names.json
  45395155  2025-04-28T19:17:23Z  gs://fashion-mnist-datasets/custom_jobs_normalized/fashion_mnist_normalized.npz
TOTAL: 3 objects, 45396319 bytes (43.29 MiB)


CompletedProcess(args=['gsutil', 'ls', '-l', 'gs://fashion-mnist-datasets/custom_jobs_normalized/'], returncode=0)

In [6]:
import subprocess

# Delete the Cloud Function
result = subprocess.run(
    ["gcloud", "functions", "delete", "fashion-mnist-normalizer", 
     "--region=us-central1", "--quiet"],
    capture_output=True,
    text=True
)

# Print the output
print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)
print(f"Return code: {result.returncode}")

if result.returncode == 0:
    print("Cloud Function successfully deleted!")
else:
    print("Error deleting Cloud Function. See error messages above.")

STDOUT: 
STDERR: Preparing function...
......done.
Deleting function...
[Service]..........................done
[Artifact Registry]done
Done.
Deleted [projects/fashion-mnist-gcp/locations/us-central1/functions/fashion-mnist-normalizer].

Return code: 0
Cloud Function successfully deleted!


***
***